
**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of the TODO's and/or use more than one cell to complete all the tasks.

In [3]:
# TODO: Install any packages that you might need
from tqdm import tqdm
import sagemaker
from sagemaker import get_execution_role

In [3]:
# TODO: Import any packages that you might need

In [4]:
session = sagemaker.Session()

bucket = session.default_bucket()
print("Default Bucket: {}".format(bucket))

region = session.boto_region_name
print("AWS Region: {}".format(region))

role = get_execution_role()
print("RoleArn: {}".format(role))

prefix = "capstone-inventory-project"

Default Bucket: sagemaker-us-east-1-453227144398
AWS Region: us-east-1
RoleArn: arn:aws:iam::453227144398:role/service-role/AmazonSageMaker-ExecutionRole-20211215T161812


## Data Preparation
**TODO:** Run the cell below to download the data.

The cell below creates a folder called `train_data`, downloads training data and arranges it in subfolders. Each of these subfolders contain images where the number of objects is equal to the name of the folder. For instance, all images in folder `1` has images with 1 object in them. Images are not divided into training, testing or validation sets. If you feel like the number of samples are not enough, you can always download more data (instructions for that can be found [here](https://registry.opendata.aws/amazon-bin-imagery/)). However, we are not acessing you on the accuracy of your final trained model, but how you create your machine learning engineering pipeline.

First, we wil download our data and split it into train, validation and test datasets.

In [5]:
number_of_images = {
    "1": 1228,
    "2": 2299,
    "3": 2666,
    "4": 2373,
    "5": 1875
}

train_size = 0.7
validation_size = 0.2
test_size = 0.1

In [6]:
import os
import json
import boto3
 
def download_and_arrange_data(dataset):     
    s3_client = boto3.client('s3')

    with open('file_list.json', 'r') as f:
        d=json.load(f)

    for k, v in d.items(): 
        # Which images for the dataset?
        if dataset == "train":
            start = 0
            end = int(number_of_images[k] * train_size)
        elif dataset == "validation":
            start = int(number_of_images[k] * train_size)
            end = start + int(number_of_images[k] * validation_size)
        elif dataset == "test":
            start = int(number_of_images[k] * (train_size+validation_size))
            end = number_of_images[k]
        else:
            print("Dataset not recognized!")
            return
            
        print(f"Downloading Images with {k} objects")
        directory=os.path.join("data", dataset, k)
        if not os.path.exists(directory):
            os.makedirs(directory)   
        
        count = 0
        for file_path in tqdm(v):
            if count <= end and count >= start:
                file_name=os.path.basename(file_path).split('.')[0]+'.jpg'
                s3_client.download_file('aft-vbi-pds', os.path.join('bin-images', file_name), os.path.join(directory, file_name))
            count += 1

In [7]:
download_and_arrange_data("train")

  0%|          | 1/1228 [00:00<02:35,  7.90it/s]

  0%|          | 1/2299 [00:00<04:37,  8.29it/s]

  0%|          | 2/2666 [00:00<03:09, 14.09it/s]

  0%|          | 1/2373 [00:00<04:43,  8.36it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

100%|██████████| 1875/1875 [02:10<00:00, 14.38it/s]


In [7]:
download_and_arrange_data("validation")

 70%|███████   | 860/1228 [00:00<00:00, 5434.03it/s]

 70%|███████   | 1610/2299 [00:00<00:00, 13851.13it/s]

 70%|███████   | 1867/2666 [00:00<00:00, 16095.17it/s]

 70%|███████   | 1663/2373 [00:00<00:00, 9532.13it/s]

 70%|███████   | 1314/1875 [00:00<00:00, 7300.79it/s]

100%|██████████| 1875/1875 [00:40<00:00, 46.70it/s]  


In [9]:
download_and_arrange_data("test")

 90%|█████████ | 1106/1228 [00:00<00:00, 9135.78it/s]

  0%|          | 0/2299 [00:00<?, ?it/s]

 90%|█████████ | 2400/2666 [00:00<00:00, 23396.85it/s]

 90%|█████████ | 2136/2373 [00:00<00:00, 21308.96it/s]

 90%|█████████ | 1688/1875 [00:00<00:00, 16241.11it/s]

100%|██████████| 1875/1875 [00:18<00:00, 104.08it/s]  


Then, we wil import the data to S3 using the aws CLI.

In [25]:
%%capture
os.environ["PREFIX"] = prefix
os.environ["DATA"] = "data"
os.environ["DEFAULT_S3_BUCKET"] = bucket
!aws s3 sync data s3://${DEFAULT_S3_BUCKET}/${PREFIX}/${DATA}/

## Dataset
**TODO:** Explain what dataset you are using for this project. Give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understanding of it. You can find more information about the data [here](https://registry.opendata.aws/amazon-bin-imagery/).

In [ ]:
#TODO: Perform any data cleaning or data preprocessing

In [ ]:
#TODO: Upload the data to AWS S3

## Model Training
**TODO:** This is the part where you can train a model. The type or architecture of the model you use is not important. 

**Note:** You will need to use the `train.py` script to train your model.

In [ ]:
#TODO: Declare your model training hyperparameter.
#NOTE: You do not need to do hyperparameter tuning. You can use fixed hyperparameter values

In [ ]:
#TODO: Create your training estimator

In [ ]:
# TODO: Fit your estimator

## Standout Suggestions
You do not need to perform the tasks below to finish your project. However, you can attempt these tasks to turn your project into a more advanced portfolio piece.

### Hyperparameter Tuning
**TODO:** Here you can perform hyperparameter tuning to increase the performance of your model. You are encouraged to 
- tune as many hyperparameters as you can to get the best performance from your model
- explain why you chose to tune those particular hyperparameters and the ranges.


In [ ]:
#TODO: Create your hyperparameter search space

In [ ]:
#TODO: Create your training estimator

In [ ]:
# TODO: Fit your estimator

In [ ]:
# TODO: Find the best hyperparameters

### Model Profiling and Debugging
**TODO:** Use model debugging and profiling to better monitor and debug your model training job.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

### Model Deploying and Querying
**TODO:** Can you deploy your model to an endpoint and then query that endpoint to get a result?

In [ ]:
# TODO: Deploy your model to an endpoint

In [ ]:
# TODO: Run an prediction on the endpoint

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done

### Cheaper Training and Cost Analysis
**TODO:** Can you perform a cost analysis of your system and then use spot instances to lessen your model training cost?

In [ ]:
# TODO: Cost Analysis

In [ ]:
# TODO: Train your model using a spot instance

### Multi-Instance Training
**TODO:** Can you train your model on multiple instances?

In [ ]:
# TODO: Train your model on Multiple Instances